## Importando as bibliotecas

In [ ]:
from difflib import SequenceMatcher

In [ ]:
import netpixi
from netpixi.integration.gt import Graph
from netpixi.integration.gt import *
import seaborn as sns
from regression.integration.gt import *

import cpnet

from graph_tool import spectral
from graph_tool import clustering

import itertools


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import regression as reg

from scipy import stats
from matplotlib import pyplot as plt

## Criando o grafo

In [ ]:
# Criando o grafo
g = Graph(directed=False) # não-dirigido

# Criando um dicionário estilo {nome do repositorio:[topicos]}
repositorio_e_topicos = {}

# Criando os vértices & preenchendo o dicionário
with open('Github_data_refatorado.csv', "r") as arquivo:
    arquivo.readline()
    for linha in arquivo:
        try:
            div = linha.split(',')
            
            top_tag = linha.split('[')[1].split(']')[0].replace("'", "").replace(" ", "").split(",")
            topic = div[2].lower()
            repositorio = div[3]
            
            if not g.has_vertex(repositorio):
                g.add_vertex(repositorio)
                
                if top_tag != ['']:
                    repositorio_e_topicos[repositorio] = top_tag
                else:
                    repositorio_e_topicos[repositorio] = [topic]
        except:
            continue

In [ ]:
# Quantidade de vértices
len(g.get_vertices())

In [ ]:
# Quantidade de tópicos diferentes
lista_distinct_topics = list(set(item for sublista in t.values() for item in sublista))
print(len(lista_distinct_topics))

In [ ]:
# Função de similaridade
def conta_similaridade(lista, lista2):
    cache = {}
    cont = 0
    for i, b in itertools.product(lista, lista2):
        if (i, b) in cache:
            cont += cache[(i, b)]
        else:
            sim = SequenceMatcher(None, i, b).ratio()
            if sim >= 0.95:
                cont += 1
            cache[(i, b)] = sim >= 0.95
    return cont

In [ ]:
quant = len(t.values())

lista_numeros = []
cont = 0
for p, b in t.items():
    print(f"{cont}/{quant-1}", end = "\r")
    for i in t:
        nummero = conta_similaridade(t[i], b)
        lista_numeros.append(nummero)
        if p != i and not g.has_edge(p, i) and nummero >= 2:
            g.add_edge(p, i)
    cont += 1

In [ ]:
lista_distinct = list(set(lista_numeros))[1:]
lista_count = []
for i in lista_distinct:
    (lista_numeros.count(i))
    lista_count.append(lista_numeros.count(i))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.bar(lista_distinct, lista_count)
plt.show()

## Salvando o grafo

In [ ]:
gt_save(g, 'git_hub.net.gz')

## Compilando e renderizando o grafo

In [ ]:
g = gt_load('git_hub.net.gz')
r = netpixi.render('git_hub.net.gz')

## Melhorando visualmento o grafo

In [ ]:
# mudar a cor de fundo do grafo
r.graph(color=0xffffff)

# mudar a cor e o tamanho dos vértices
r.vertex_default(color=0x000000, size=10)

# mudar um vértice específico a partir de seu id
r.vertex(0, color=0xff0000, size=20)
r.vertex(1, color=0xff0000, size=20)

# mudar a cor, a espessura e a curvatura das arestas
r.edge_default(color=0x000000, width=1, curve1=1, curve2=1)

## Análise exploratória do grafo

In [ ]:
# Números de vértices

n = g.num_vertices()
n

In [ ]:
# Número de arestas

m = g.num_edges()
m

In [ ]:
# Número máximo possível de arestas

if g.is_directed():
    max_edges = n * (n - 1)
else:
    max_edges = n * (n - 1) // 2
max_edges

In [ ]:
# Densidade

d = m / max_edges
d

In [ ]:
# Propriedades de cada vértice

data = gt_data(g)

# Degree

degrees = []
for v in g.all_vertices():
    degrees.append(v.total_degree())
    
data['degree'] = degrees
data

In [ ]:
# Histograma do degree

sns.histplot(data['degree']);
data['degree'].describe()

## Limpando o grafo

In [ ]:
g = gt_clean(g)
m = gt_draw.sfdp_layout(g)
gt_move(g, m)
gt_save(g, 'git_hub.net.gz')
netpixi.render('git_hub.net.gz', infinite=True);

In [ ]:
gt_clean(g)
m = gt_draw.fruchterman_reingold_layout(g)
gt_move(g, m)
gt_save(g, 'git_hub.net.gz')
netpixi.render('git_hub.net.gz');

# Análise das variáveis dependentes e independentes

In [ ]:
data

In [ ]:
lc = clustering.local_clustering(g)

clustering = []
for value in lc:
    clustering.append(value)

In [ ]:
g1 = gt_load('git_hub.net.gz')
gt_data(g1)

In [ ]:
for i in g:
    print(i["props"])

In [ ]:
result = reg.linear(data=data, formula='commits ~  + star')
result.micro_summary()